In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
import acquire as acq
import prepare as prep

In [3]:
total_df = prep.prep_telco(acq.get_telco_data())

In [4]:
#acq.peekatdata(df)
#df.head()

In [8]:
total_df.shape

(7032, 34)

In [9]:
total_df.columns

Index(['payment_type_id', 'internet_service_type_id', 'contract_type_id',
       'customer_id', 'gender', 'senior_citizen', 'partner', 'dependents',
       'tenure', 'phone_service', 'multiple_lines', 'online_security',
       'online_backup', 'device_protection', 'tech_support', 'streaming_tv',
       'streaming_movies', 'paperless_billing', 'monthly_charges',
       'total_charges', 'churn', 'contract_type', 'internet_service_type',
       'payment_type', 'churn_encode', 'phone_id', 'movies_encode',
       'tv_encode', 'streaming_services', 'online_security_encode',
       'online_backup_encode', 'online_security_backup', 'household_type_id',
       'gender_encode'],
      dtype='object')

In [6]:
df = total_df

In [7]:
df.total_charges.sort_values()

761       100.2
3103     100.25
6574     100.35
963       100.4
3978      100.8
5775      100.9
4169     1001.2
6298     1001.5
829     1003.05
6932    1004.35
6363     1004.5
851     1004.75
6443     1005.7
6633     1006.9
6182     1007.8
6330     1007.9
3534    1008.55
3558     1008.7
6875     1008.7
350     1009.25
422       101.1
3608     101.45
1064     101.65
5979      101.9
1505       1010
253     1011.05
1074     1011.5
2623     1011.8
6375     1012.4
6392    1013.05
         ...   
6685     982.95
407      985.05
4005     987.95
6975     987.95
6390     989.05
6125     989.05
1340     989.45
6132      99.45
6310       99.6
903       99.75
2436      99.75
593       99.95
2667      990.3
6381     990.45
577      990.85
4227      990.9
553       992.7
6059     993.15
913      994.55
3781      994.8
347      995.35
5539     996.45
770      996.85
6308     996.95
3797     997.65
6341     997.75
6261      998.1
3702     999.45
3392      999.8
4349      999.9
Name: total_charges, Len

## Split into X's, y's and Train/Tests

In [ ]:
X = df[['household_type_id','internet_service_type_id','gender']]
y = df[['churn_encode']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state = 123)

## Logistic Regression fitting

In [ ]:
logit = LogisticRegression()
logit.fit(X_train, y_train)
y_pred = logit.predict(X_train)
y_pred_proba = logit.predict_proba(X_train)

In [ ]:
print('Columns: \n:', X_train.columns)
print('Coefficient: \n', logit.coef_)
print('Intercept: \n', logit.intercept_)

In [ ]:
y_pred

#### Probability of Stay vs Churn 

In [ ]:
y_pred_proba

#### Accuracy 

In [ ]:
(y_train['churn_encode'] == y_pred).sum() / y_train.shape[0]